# Программирование на Python
## Семинар 5. Повторение основ

Имеются данные по сотрудникам некоторой Компании. Они устроены следующим образом. Компания подразделяется на Блоки, Блоки - на Департаменты. Название Департамента необязательно уникально. От отдела кадров по результатам некоторого тестирования вам пришли файлы со статистиками по сотрудникам. Они находятся в папке `data`.

Также существует справочник электронных почт руководителей Департаментов и шаблон письма (папка `sources`).

#### Задача

- изучить файлы от отдела кадров, понять общую структуру;
- на их основании определить среднее значение по каждой из статистик в разрезе Департамента (и проинтерпретировать его - выше или ниже среднего значения по всей компании);
- составить по результатам письмо руководителю каждого из Департаментов;
- автоматически разослать его, используя ваш корпоративный аккаунт.

In [1]:
from pathlib import Path  # для ООП-работы с файловой системой
import openpyxl
from email.mime.text import MIMEText  # для создание email'а по протоколу MIME (для дальнейшей передачи)
from email.mime.multipart import MIMEMultipart
import smtplib  # для SMTP подключения
from configparser import ConfigParser  # для парсинга конфигов
from time import sleep  # для задержки при отправке

In [ ]:
# parse info -> dict with key (block, department)
# parse_file -> dict with key (block, department)
# iterate through filenames, apply parse_file function -> collect to dict
# unite data for value1, value2 -> compute total mean
# create email text
# send email

In [2]:
# constants.py
DATA_PATH_STR = 'data'
SOURCE_PATH_SOURCE = 'sources'
COLNAMES_MAPPER = {
    'ID': 'id',
    'Показатель 1': 'value1',
    'Показатель 2': 'value2',
}
INTERP_MAPPER = {
    True: 'показатель ниже среднего по компании',
    False: 'показатель не ниже среднего по компании',
}
NDIGITS = 2

In [3]:
# functions.py
def load_info(path: Path) -> dict:
    wb = openpyxl.load_workbook(path)
    ws = wb.active

    info_dict = {
        (row[0].value, row[1].value): {'fio': row[2].value, 'email': row[3].value}
        for row
        in ws.iter_rows()
    }

    wb.close()

    _ = info_dict.pop(('Блок', 'Департамент'))

    return info_dict

def load_department(path: Path) -> dict:
    block = path.parent.name
    department_name = path.name.strip('.xlsx')
    
    wb = openpyxl.load_workbook(path)
    ws = wb.active

    department = {}

    for col in ws.iter_cols():
        values = [cell.value for cell in col if cell.value is not None]
        colname = COLNAMES_MAPPER.get(values[0], 'UNKNOWN_COLUMN')
        colvalues = values[1:]
        
        department.update({colname: colvalues})

    wb.close()

    return {(block, department_name): department}

def mean(vector) -> float:
    return sum(vector) / len(vector)

In [11]:
# main.py
data_path = Path(DATA_PATH_STR)
source_path = Path(SOURCE_PATH_SOURCE)
info_path = source_path / 'Справочник.xlsx'
template_path = source_path / 'message_template.html'
config_path = source_path / 'config.ini'
filenames = data_path.glob('Блок */Департамент *.xlsx')

# load template
with open(template_path, 'r') as file:
    template = file.read()

# load configs
cp = ConfigParser()
_ = cp.read(config_path)
smtp_configs = cp['smtp.client']

# load info
info = load_info(info_path)

# save all tables
datas = {}

value1 = []
value2 = []

for path in filenames:
    data = load_department(path)

    datas.update(data)

    for dct in data.values():
        value1.extend(dct['value1'])
        value2.extend(dct['value2'])

value1_mean = mean(value1)
value2_mean = mean(value2)

In [17]:
server = smtplib.SMTP_SSL(smtp_configs['host'], smtp_configs['port'])
server.login(smtp_configs['login'], smtp_configs['password'])

for key, value in info.items():
    data = datas.get(key)
    
    if data is None:
        continue

    format_dict = {}
    
    # add fio, block, department
    format_dict.update(
        {
            'fio': value['fio'],
            'block': key[0],
            'department': key[1],
        }
    )

    # add other params
    value1_mean_ = mean(data['value1'])
    value2_mean_ = mean(data['value2'])

    interp1 = INTERP_MAPPER[value1_mean_ < value1_mean]
    interp2 = INTERP_MAPPER[value2_mean_ < value2_mean]

    format_dict.update(
        {
            'value1': round(value1_mean_, NDIGITS),
            'value2': round(value2_mean_, NDIGITS),
            'interp1': interp1,
            'interp2': interp2,
        }
    )

    # create message text
    text = template.format(**format_dict)

    # create email
    email = MIMEMultipart()
    email['From'] = smtp_configs['login']
    email['To'] = value['email']
    email['Subject'] = f'Отчет по блоку {format_dict["block"]}, департаменту {format_dict["department"]}.'

    message_text = MIMEText(text, 'html')
    
    email.attach(message_text)

    # send email
    send_response = server.sendmail(email['From'], email['To'], email.as_string())

    if send_response == {}:
        print(f'Report for the head of block {format_dict["block"]}, department {format_dict["department"]} successfully sent!')
    else:
        print('Something went wrong:(')

    # sleep after each sending
    sleep(1)

server.close()

Report for the head of block Блок 1, department Департамент A successfully sent!
Report for the head of block Блок 1, department Департамент B successfully sent!
Report for the head of block Блок 1, department Департамент C successfully sent!
Report for the head of block Блок 2, department Департамент A successfully sent!
Report for the head of block Блок 2, department Департамент B successfully sent!
Report for the head of block Блок 2, department Департамент C successfully sent!
Report for the head of block Блок 2, department Департамент D successfully sent!
Report for the head of block Блок 2, department Департамент E successfully sent!


KeyboardInterrupt: 

In [22]:
list_of_lists = [[1, 2], [3, 4]]  # -> [1, 2, 3, 4]

In [26]:
sum(list_of_lists, [])

[1, 2, 3, 4]

In [23]:
[value for lst in list_of_lists for value in lst]

[1, 2, 3, 4]

In [ ]:
for key in info.keys():
    data = datas.get(key)
    
    if data is None:
        continue

    
    # get data from datas by key (if possible) else continue
    # extract mean1, mean2, interp1, interp2 from data
    # extract fio, email from info
    # extract block, department from key

In [70]:
datas

{('Блок 1',
  'Департамент A'): {'ID': ['f660317c-6223-41fb-b131-b2b34e10767c',
   '2e8b6ace-9383-49dd-8214-d76c6b9649f2',
   'd855ad41-e5fa-4f85-80f8-07e037523597',
   'e5329fc2-7c32-4caa-8aad-0cdfd63c26a7',
   '6137cf2d-23ea-4413-bb1e-d24351fc28f0',
   '5c745038-6192-4eb6-834d-8ad54393229f',
   '57135763-fa5c-41b7-bff9-1e4d22ef91a1',
   'bfd4cb5e-a4dd-44bd-9eb0-966e8600d1d6',
   'bae1c82e-c47e-460f-88bf-220b64c78b3d',
   '31fa27df-e77c-47f7-b8e1-f2807be0a31f',
   'c51f8ba2-c138-4361-9db7-1f8b47b837c9',
   'e359a63d-ed95-4a3d-acc2-316c6e14f7d8',
   '21ef00f0-766f-468e-b163-50dcc8537a02',
   '2b218c55-1496-48d0-80d6-14f63a37eac8',
   '20e2beb0-3a57-47b0-adee-2a4586f1b167',
   '29975235-8164-4a7c-a46a-9e456b270ddd',
   '6964ae31-4eb5-423a-91cb-e2723ad603e5',
   'a61fbdc9-e17a-468c-8ac7-c28fd6aa5db9',
   '3f16c43f-43fb-4b6c-9919-132a327f2597',
   'a76d6e30-4ab5-4943-ba10-e0291c3bea30',
   '4cb63052-99ae-4175-b360-fbdef69262f7',
   '165b1c17-bcd8-47ca-bfcf-710f91c53e6f',
   '3cc0d328-7e6c

In [ ]:
for 

In [59]:
value1

[7.5,
 3.75,
 3,
 3.325,
 2.875,
 3.425,
 3.15,
 5.225000000000001,
 4.45,
 2.35,
 5.149999999999999,
 4.175,
 6.375,
 2.2,
 5.75,
 8.475000000000001,
 5.4,
 2,
 5.15,
 5.699999999999999,
 4.7,
 3.9,
 4.449999999999999,
 6.75,
 3,
 5.6,
 2.55,
 5.875,
 2.4,
 2.675,
 4.15,
 3.275,
 3.3,
 4.475,
 2.725,
 4.725000000000001,
 4.225000000000001,
 6.524999999999999,
 3.625,
 4.575,
 6.7,
 3.4,
 4.625,
 4.4,
 3.425,
 4.525,
 2.975,
 4.575,
 3.425,
 3.75,
 7.35,
 4.100000000000001,
 3.95,
 6.050000000000001,
 5.7,
 7.2,
 5.4,
 3.775,
 5.175,
 2.225,
 5.925000000000001,
 3.95,
 7.7,
 4.05,
 3.775,
 3,
 2.65,
 5.35,
 4.925,
 2.05,
 6,
 4,
 2.525,
 4.95,
 1.05,
 2.875,
 5.625,
 6.25,
 3.3,
 3.2,
 2.2,
 3.475000000000001,
 4.45,
 4.5,
 6.5,
 5.75,
 2.825,
 5.100000000000001,
 3.5,
 3.575,
 5.175000000000001,
 6.475,
 6.675000000000001,
 4.775,
 4.825,
 6.149999999999999,
 2.8,
 3.525,
 3.225,
 5.425,
 4.95,
 4.525,
 4.975,
 1.725,
 6.375,
 3.65,
 3.325,
 4,
 2.075,
 3.775,
 6.825,
 5.775,
 5.67499

In [ ]:
# load load_department
# load info
# join load_department with info
# insert value into template
# create email, attach text and send e,ail

In [52]:
path1.parent.name

'Блок 1'

In [54]:
load_department(path1)

{('Блок 1',
  'Департамент A'): [{'ID': ['f660317c-6223-41fb-b131-b2b34e10767c',
    '2e8b6ace-9383-49dd-8214-d76c6b9649f2',
    'd855ad41-e5fa-4f85-80f8-07e037523597',
    'e5329fc2-7c32-4caa-8aad-0cdfd63c26a7',
    '6137cf2d-23ea-4413-bb1e-d24351fc28f0',
    '5c745038-6192-4eb6-834d-8ad54393229f',
    '57135763-fa5c-41b7-bff9-1e4d22ef91a1',
    'bfd4cb5e-a4dd-44bd-9eb0-966e8600d1d6',
    'bae1c82e-c47e-460f-88bf-220b64c78b3d',
    '31fa27df-e77c-47f7-b8e1-f2807be0a31f',
    'c51f8ba2-c138-4361-9db7-1f8b47b837c9',
    'e359a63d-ed95-4a3d-acc2-316c6e14f7d8',
    '21ef00f0-766f-468e-b163-50dcc8537a02',
    '2b218c55-1496-48d0-80d6-14f63a37eac8',
    '20e2beb0-3a57-47b0-adee-2a4586f1b167',
    '29975235-8164-4a7c-a46a-9e456b270ddd',
    '6964ae31-4eb5-423a-91cb-e2723ad603e5',
    'a61fbdc9-e17a-468c-8ac7-c28fd6aa5db9',
    '3f16c43f-43fb-4b6c-9919-132a327f2597',
    'a76d6e30-4ab5-4943-ba10-e0291c3bea30',
    '4cb63052-99ae-4175-b360-fbdef69262f7',
    '165b1c17-bcd8-47ca-bfcf-710f91c53e

In [41]:
wb = openpyxl.load_workbook(path1)
ws = wb.active

# path1

In [43]:
for col in ws.iter_cols():
    print([cell.value for cell in col])

['ID', None, 'f660317c-6223-41fb-b131-b2b34e10767c', '2e8b6ace-9383-49dd-8214-d76c6b9649f2', 'd855ad41-e5fa-4f85-80f8-07e037523597', 'e5329fc2-7c32-4caa-8aad-0cdfd63c26a7', '6137cf2d-23ea-4413-bb1e-d24351fc28f0', '5c745038-6192-4eb6-834d-8ad54393229f', '57135763-fa5c-41b7-bff9-1e4d22ef91a1', 'bfd4cb5e-a4dd-44bd-9eb0-966e8600d1d6', 'bae1c82e-c47e-460f-88bf-220b64c78b3d', '31fa27df-e77c-47f7-b8e1-f2807be0a31f', 'c51f8ba2-c138-4361-9db7-1f8b47b837c9', 'e359a63d-ed95-4a3d-acc2-316c6e14f7d8', '21ef00f0-766f-468e-b163-50dcc8537a02', '2b218c55-1496-48d0-80d6-14f63a37eac8', '20e2beb0-3a57-47b0-adee-2a4586f1b167', '29975235-8164-4a7c-a46a-9e456b270ddd', '6964ae31-4eb5-423a-91cb-e2723ad603e5', 'a61fbdc9-e17a-468c-8ac7-c28fd6aa5db9', '3f16c43f-43fb-4b6c-9919-132a327f2597', 'a76d6e30-4ab5-4943-ba10-e0291c3bea30', '4cb63052-99ae-4175-b360-fbdef69262f7', '165b1c17-bcd8-47ca-bfcf-710f91c53e6f', '3cc0d328-7e6c-49c6-ae6c-da276fe956e0', 'c0347f2e-eb2e-4ae6-8c71-0dd4685a7843', 'e8469347-bd59-4999-b452-77

In [44]:
wb.close()